In [1]:
import requests

API_KEY = "vyLy4J7duzWtk0nU5rBKWp87OUHHC3AW72av8i02"
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-01-01&end_date=2023-01-07&api_key={API_KEY}"

response = requests.get(url)
data = response.json()

# Extract relevant features
asteroids = []
for date in data["near_earth_objects"]:
    for asteroid in data["near_earth_objects"][date]:
        asteroids.append({
            "size": asteroid["estimated_diameter"]["kilometers"]["estimated_diameter_max"],
            "velocity": asteroid["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"],
            "distance": asteroid["close_approach_data"][0]["miss_distance"]["kilometers"],
            "hazardous": asteroid["is_potentially_hazardous_asteroid"]
        })

print(asteroids)


[{'size': 1.7621382518, 'velocity': '46527.0978301312', 'distance': '33403488.692868118', 'hazardous': True}, {'size': 0.744800533, 'velocity': '20959.8207659374', 'distance': '39565961.760205039', 'hazardous': False}, {'size': 1.2474983468, 'velocity': '58249.6896629513', 'distance': '39330824.516289241', 'hazardous': False}, {'size': 0.0517654482, 'velocity': '24703.7439103688', 'distance': '8526777.284930033', 'hazardous': False}, {'size': 0.0682401509, 'velocity': '25881.1975863978', 'distance': '64975962.736835246', 'hazardous': False}, {'size': 0.4508582062, 'velocity': '86987.5810714278', 'distance': '15384366.923158605', 'hazardous': True}, {'size': 0.0342010925, 'velocity': '29406.9410907668', 'distance': '27322012.490044019', 'hazardous': False}, {'size': 0.0313357417, 'velocity': '20200.2112060543', 'distance': '33343158.085945594', 'hazardous': False}, {'size': 0.0411187571, 'velocity': '22559.4578038017', 'distance': '39470300.39992781', 'hazardous': False}, {'size': 0.019

In [2]:
import pandas as pd

df = pd.DataFrame(asteroids)

# Convert numeric values to float
df["size"] = df["size"].astype(float)
df["velocity"] = df["velocity"].astype(float)
df["distance"] = df["distance"].astype(float)
df["hazardous"] = df["hazardous"].astype(int)

print(df.head())


       size      velocity      distance  hazardous
0  1.762138  46527.097830  3.340349e+07          1
1  0.744801  20959.820766  3.956596e+07          0
2  1.247498  58249.689663  3.933082e+07          0
3  0.051765  24703.743910  8.526777e+06          0
4  0.068240  25881.197586  6.497596e+07          0


In [3]:
from sklearn.model_selection import train_test_split

X = df[["size", "velocity", "distance"]]
y = df["hazardous"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 95.65%


In [5]:
new_asteroid = [[0.5, 25000, 500000]]  # Example: Size=0.5km, Velocity=25,000 km/h, Distance=500,000 km
prediction = model.predict(new_asteroid)
print("Hazardous" if prediction[0] == 1 else "Not Hazardous")


Not Hazardous


C:\Users\Owner\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [6]:
import pickle
# Save model
with open("asteroid_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model trained and saved as asteroid_model.pkl")

Model trained and saved as asteroid_model.pkl
